In [1]:
from transformers import MistralForCausalLM

c:\Users\Stephan\anaconda3\envs\chatbot-qa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MistralForCausalLM.from_pretrained("VAGOsolutions/SauerkrautLM-7b-v1-mistral")

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.20s/it]


In [4]:
model.model

MistralModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x MistralDecoderLayer(
      (self_attn): MistralAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MistralRotaryEmbedding()
      )
      (mlp): MistralMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): MistralRMSNorm()
      (post_attention_layernorm): MistralRMSNorm()
    )
  )
  (norm): MistralRMSNorm()
)

IMPORTANT:

https://huggingface.co/docs/transformers/model_doc/mistral

# Verwendete Modelle

## Modellarchitektur LLama2 13b

Ressourcen:
- LLaMA explained ([Umar Jamil](https://www.youtube.com/watch?v=Mn_9W1nCFLo))
- LLaMA: Open and Efficient Foundation Language Models ([Hugo Touvron et al.](https://arxiv.org/abs/2302.13971))
- Llama 2: Open Foundation and Fine-Tuned Chat Models ([Hugo Touvron et al.](https://arxiv.org/abs/2307.09288))

![title](img/llama2-architecture.png)

## Modellarchitektur Mistral 7b

## Unterschiede Vorteile / Nachteile

## Weshalb wurden diese Modelle verwendet

# Methodik

## Ziel

Unser Ziel ist es, ein LLM zu entwickeln, welches den Question Answering Pfad unseres Chatbots `Data` übernehmen kann. Dazu möchten wir ein existierendes LLM so fine-tunen, dass es gestellte Fragen auf zur Verfügung gestelltem Kontext beantworten kann (`Retrieval Augmented Generation`). Dabei soll das LLM erkennen, ob die zur Beantwortung der Frage notwendigen Informationen im Kontext vorhanden sind, und wenn nicht, die Beantwortung ablehnen (kein pre-trained knowledge verwenden) und diesen Fakt in der Antwort kommunizieren. Der Kontext besteht dabei aus einer Liste von relevanten Chunks, die durch Retrieval aus unserer Vektordatenbank ausgelesen werden. Die Chunks wurden aus den Spaces Daten des Studiengangs Data Science generiert, und enthalten neben dem Text Metadaten wie das Modulkürzel, die vollständige Bezeichnung, die Anzahl ECTS und die Quelle. Das LLM soll in seinen generierten Antworten die Quelle des Chunks angeben, aus dem Informationen zur Beantwortung der Frage verwendet wurden. Damit können Studenten, die den Bot benutzen, bei Bedarf die Antworten des LLMs überprüfen.

## Fine-tuning

Fine-tuning bezeichnet den Prozess der Anpassung eines bereits vortrainierten Modells (pre-trained model) auf eine spezifische Aufgabe oder Daten, um die Leistungsfähigkeit des Modells für diese Aufgabe zu verbessern. Hierbei wird die generelle Wissensbasis des Modells beibehalten, während es gleichzeitig für die speziellen Anforderungen und Nuancen des neuen Anwendungsbereichs optimiert wird.

### Pre-trained Models 

Modelle wie Llama 2 und Mistral sind vortrainierte Sprachmodelle, die ein breites Verständnis von Sprache und Wissen bieten. Indem wir solche Modelle als Ausgangspunkt nehmen, können wir von dem bereits erworbenen Wissen und den Fähigkeiten des Modells profitieren. Das spart viel Zeit als auch Ressourcen, die wir im Rahmen von NPR gar nicht hätten. Mittels Fine-tuning passen wir diese Modelle an die unter Ziel genannten Bedürfnisse und Daten des Projekts an. Beide Modelle sind auf der Hugging Face Plattform verfügbar. Die Hugging Face Bibliothek `transformers` bietet uns eine konsistente Schnittstelle zum Arbeiten mit verschiedenen Sprachmodellen.

Wir haben die folgenden Modelle und Datasets von Hugging Face verwendet:

- Used model (https://huggingface.co/flozi00/Llama-2-13b-german-assistant-v7) | Download: 22.12.2023
- Used model (https://huggingface.co/meta-llama/Llama-2-13b-hf) | Download: 23.12.2023
- Used model (https://huggingface.co/VAGOsolutions/SauerkrautLM-7b-v1-mistral) | Download: 29.12.2023
- Fine-Tuning Dataset Base (https://huggingface.co/datasets/deepset/germanquad) | Download: 22.12.2023, Abstractive dataset modification: 22.12.2023

### Quantization und LoRA

Beim Fine-tuning müssen verschiedene Faktoren berücksichtigt werden, wie die Modellgrösse im Verhältnis zur verfügbaren GPU-Kapazität. Grössere Modelle benötigen mehr Speicher und Rechenleistung, was die Hardwareanforderungen erhöht und Kosten sowie Zugänglichkeit beeinflusst. Uns steht ein Rechner von Tobias mit einer `NVIDIA GeForce RTX 4090` zur Verfügung. Ohne die Anwendung einiger Tricks wäre es uns nicht möglich, die ausgewhählten 7B und 13B Parameter grossen Modelle zu fine-tunen.

Quantization und LoRA (Low-Rank Adaptation) sind Techniken, die helfen, die Effizienz von LLMs zu verbessern, indem sie die Modellgrösse reduzieren oder die Anpassungsfähigkeit erhöhen, ohne signifikante Verluste in der Leistung zu verursachen. Quantization verringert den Speicherbedarf, indem es die Präzision der Gewichte des Modells reduziert. LoRA hingegen ist eine Methode, die die Anzahl der zu trainierenden Parameter reduziert, indem es die Matrizen des Modells in niedrigere Ränge zerlegt.

TODO: weiter ausführen
- fine-tuning benötigt nochmal deutlich mehr memory wie inference

#### Quantization erklärt

Quantization ist der Prozess der Diskretisierung kontinuierlicher Verteilungswerte in diskrete Werte. Dies bedeutet, dass die kontinuierlichen Gewichte des Modells auf eine festgelegte Anzahl von Werten reduziert werden, was den Speicherbedarf und die Rechenzeit verringert.

<img src="./img/Quantization.png" width="300px"></img>

**Motivation:** Der Hauptantrieb hinter Quantization ist die Optimierung der Leistung pro genutztem Bit im Modell. Durch die Reduzierung der Präzision der Gewichte eines Modells können wir eine höhere Effizienz in Speicher und Berechnungen erreichen, was besonders auf Hardware mit begrenzten Ressourcen wie unserer GPU von Bedeutung ist.

**4-bit integer Quantization:** In unserem Fall verwenden wir 4-bit integer Quantization, was bedeutet, dass jedes Gewicht in unserem Modell auf eines von \(2^4 = 16\) möglichen Werten reduziert wird. Dies bietet einen guten Kompromiss zwischen Modellleistung und Speichereffizienz.

**Blockweise Quantisierung:** Eine weitere Optimierungstechnik ist die blockweise Quantisierung, bei der Eingabetensoren in kleinere Blöcke unterteilt werden, die unabhängig voneinander quantisiert werden. Dies ermöglicht eine parallele Verarbeitung über Kerne hinweg, was zu einer schnelleren Optimierung und einer hochpräzisen Quantisierung führt.

**Relevante Hyperparameter:** Neben der Anzahl der Bits gibt es weitere Variablen, die die Quantisierung verbessern können:

- **Blockgrösse:** Je kleiner die Blockgrösse, desto besser können wir unabhängige Quantisierungen erreichen und so die Genauigkeit erhöhen.
- **Datentypen:** Auch die Wahl des Datentyps macht einen Unterschied. NormalFloat4 (NF4) zum Beispiel, ist ein informationstheoretisch optimaler Datentyp für Normalverteilungen und kann einen erheblichen Unterschied in der Leistungsfähigkeit machen.

#### LoRA erklärt

Die Kernidee hinter LoRA ist, dass die Änderungen, die während des Fine-tunings an den Gewichten eines Modells vorgenommen werden, durch zwei Matrizen niedriger Rangordnung dargestellt werden können. Dies basiert auf der Annahme, dass es eine lineare Abhängigkeit in den Anpassungen gibt, und folglich kann das Fine-tuning effizienter gemacht werden, indem der Rang dieser Anpassungen reduziert wird.

<img src="./img/LoRA.png" width="200px"></img>

Technisch gesehen funktioniert LoRA, indem es eine Änderungsmatrix $\triangle{W}$ der Dimensionen $d \times k$ berechnet. Diese Änderungsmatrix wird dann zu den ursprünglichen Modellgewichten hinzugefügt, um sie zu aktualisieren. Die Matrix $\triangle{W}$ wird durch die Multiplikation zweier neuer Matrizen $B$ und $A$ berechnet: $\triangle{W} = B \times A$. Die Dimensionen von $A$ und $B$ sind so gewählt, dass die resultierende Matrix die Dimensionen der Modellgewichte hat, also beispielsweise $d \times r$ für $B$ und $r \times k$ für $A$, wobei $r$ der Rang ist.

Die Matrix $A$ wird in der Regel aus einer Gaussschen Verteilung initialisiert, während $B$ mit Nullen initialisiert wird. Während des Fine-tunings werden dann diese Gewichte durch Backpropagation angepasst, wobei nur die relativ wenigen Parameter in den Matrizen $A$ und $B$ und nicht die gesamten Gewichte des Modells aktualisiert werden.

Einer der Hauptvorteile von LoRA ist, dass es die Parameter Effizienz während des Fine-tunings erheblich verbessert. Anstatt alle Gewichte eines grossen Modells zu aktualisieren, was oft Millionen oder gar Milliarden von Parametern umfasst, werden nur die Gewichte in den niedrigrangigen Matrizen $A$ und $B$ angepasst. Dies führt zu einer drastischen Reduzierung der Anzahl der zu trainierenden Parameter und damit zu einem effizienteren und schnelleren Fine-tuning.

Obwohl LoRA standardmässig auf das Attention Module (insbesondere die Projektionen für Query und Value) angewendet wird (Hu et al., 2021), empfiehlt sich eine Anwendung auf alle Linearen Layer in den Transformer Blöcken (Dettmers et al., 2023), um die volle Leistungsfähigkeit zu erreichen. Die genaue Grösse der Adapter oder die Rangordnung der Matrizen $A$ und $B$ spielt dabei eine untergeordnete Rolle; entscheidend ist vielmehr die Fähigkeit, die relevanten Änderungen während des Fine-tunings effizient zu repräsentieren und zu optimieren.

### QLoRa

QLoRa kombiniert Quantization und LoRA, und bringt dadurch die beiden Optimierungen zusammen. Im besten Fall bleibt die Performance des Modells sogar gleich wie sie es bei einem vollständigen Fine-tuning wäre (Dettmers et. al., 2023). Insgesamt lässt sich mit QLoRA beispielsweise ein 65B Parameter grosses Modell, welches ursprünglich 780 GB GPU Memory benötigt hätte (ohne Berücksichtigung der Activations während der Backpropagation), auf nur einer grossen Datacenter GPU trainieren. In unserem Fall bedeutet dass, dass wir damit 13B und 7B Modelle wie Llama 2 und Mistral auf der 4090 GPU von Tobias Fine-tunen können.

<img src="./img/QLoRA.png" width="500px"></img>

### Datenset

TODO

### Training

Wir haben `QLoRA` mit der Hugging Face `transformers` Library implementiert. Hier sind die Details unserer Fine-tuning Implementierung:

#### LoRA-Konfiguration:

Wir haben `LoRAConfig` mit spezifischen Parametern eingerichtet:

- `r: 12` - Der Rang für unsere Matrizen A und B, eine kritische Größe, die die Anzahl der Parameter, die während des Fine-tunings angepasst werden, bestimmt.
- `lora_alpha: 10` - Ein Multiplikationsfaktor für LoRA-Gewichte, der die Skalierung der LoRA-Updates steuert.
- `lora_dropout: 0.1` - Dropout-Rate, um Overfitting während des Trainings zu vermeiden.
- `bias: "none"` - Wir haben uns entschieden, keinen Bias in LoRA zu verwenden.
- `task_type: "CAUSAL_LM"` - Unser Modell ist auf kausale Sprachmodellierung ausgerichtet.

#### BitsAndBytes-Konfiguration:

Um die Quantisierungseffizienz zu maximieren, haben wir `BitsAndBytesConfig` eingeführt:

- `load_in_4bit: True` - Aktiviert die 4-Bit Quantisierung beim Laden von Modellen.
- `bnb_4bit_use_double_quant: True` - Nutzt doppelte Quantisierung für verbesserte Genauigkeit bei 4-Bit Werten.
- `bnb_4bit_quant_type: "nf4"` - Spezifiziert den Normal-Float-4-Bit Quantisierungstyp, den wir verwenden.
- `bnb_4bit_compute_dtype: torch.bfloat16` - Legt den Datentyp für die Berechnungen fest, um einen guten Kompromiss zwischen Leistung und Genauigkeit zu gewährleisten.

#### Trainingseinstellungen:

Unser Training wird durch die `TrainingArguments` und den `SFTTrainer` gesteuert:

- `output_dir`, `overwrite_output_dir`, `per_device_*_batch_size`, und andere Standardparameter steuern, wo und wie das Training durchgeführt wird.
- `gradient_accumulation_steps` und `learning_rate` sind fein abgestimmt, um die Effektivität unseres Trainings zu maximieren.
- `logging_steps`, `num_train_epochs`, und `lr_scheduler_type` sind entscheidend für das Monitoring und die Steuerung des Trainingsverlaufs.
- `evaluation_strategy` und `save_strategy` sind auf "steps" gesetzt, was bedeutet, dass wir regelmässig evaluieren und speichern, um den Fortschritt zu überwachen.
- `load_best_model_at_end: True` und `metric_for_best_model: "loss"` stellen sicher, dass wir am Ende das Modell mit der besten Leistung auswählen.

Der `SFTTrainer` wird mit unserem Modell, Tokenizer, Daten und spezifischen `peft_config` für LoRA eingerichtet. Zusätzlich haben wir eine `DataCollatorForCompletionOnlyLM` eingerichtet, die speziell auf unser Training abgestimmt ist, falls wir nur auf Vervollständigungen trainieren wollen.

## Evaluation

### Quantitative Analyse

### Qualitative Analyse

#### Protokoll

### Fehleranalyse

# Eda Datenset

Analyse

# Evaluationsmetriken

## Quantitative Analyse

Bleu / Rouge

## Qualitative Analyse

Factuality / Coverage

# Quantitiative Analyse

Eval loss / Bleu / Rouge

Wandb report

# Qualitative Analyse

Spaces Kontext / Frage / Anwort / Factuality / Coverage 

# Fehleranalyse / Korrekturversuch

Wo wurden Fehler gemacht?

ROME

Attention scores